In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wid
import logging
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from sklearn import metrics
from IPython.display import display
from ipywidgets import HBox, Box, VBox
dataset = pd.read_csv("clean2.csv")
# create new data columns
dataset['Gls90'] = (dataset['Gls'] / dataset['90s']) # Goals per 90 minutes
dataset['GlsAst90'] = ((dataset['Gls'] + dataset['Ast']) / dataset['90s']) # Goals and assists per 90 minutes
dataset['GlsAst'] = (dataset['Gls'] + dataset['Ast'])
# log access
logging.basicConfig(filename='log.txt', format='%(asctime)s %(message)s')
logging.warning(' accessed.')

In [ ]:
# dataset cleaning, remove weird characters from player string
#dataset = dataset[dataset['Pos'] != 'DF'] # Remove Defenders
#dataset = dataset[dataset['Pos'] != 'GK'] # Remove Goalkeepers
#dataset = dataset.drop(dataset[(dataset.Min < 1000) & (dataset.Gls == 0)].index) # remove unhelpful values
dataset['Player'] = dataset['Player'].apply (lambda row: row[:-9])
dataset = dataset.fillna(0) # remove NAN

In [ ]:
dataset = dataset.drop(['Age'], axis = 1)
# dataset.corr()['GlsAst90']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(dataset[['Min', 'Gls', 'Ast']], dataset.GlsAst90, test_size=0.2, random_state=1)
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(x_train)
# X_test = sc.transform(x_test)

Player performances with respect to time played through a season.
Min = Minutes
GlsAst = Total of both goals and assists for a player
Gls90 = The amount of goals a player scores per 90 minutes (One match)
GlsAst90 = The amount of goals and assists a player contributes per 90 minutes
90s = The total number of minutes played divided by 90, 34 would mean every minute of the 34 game season played

In [ ]:
fig, axs = plt.subplots(ncols=4, figsize=(30,5))
sns.scatterplot(data = dataset, x = "Min", y = "GlsAst", hue = 'xG', ax=axs[0])
sns.scatterplot(data = dataset, x = 'Min', y = 'Gls90', hue = 'Gls', palette = 'magma', ax=axs[1])
sns.scatterplot(data = dataset, x = 'Min', y = 'GlsAst90', hue = 'Gls', palette = 'magma', ax=axs[2])
sns.scatterplot(data = dataset, x = 'Gls', y = '90s', ax=axs[3]);

In [ ]:
model = BayesianRidge()

model.fit(x_train, y_train);

In [ ]:
# text = wid.Text()
# display(text)

# def handle_submit(sender):
#     print(text.value)

# text.on_submit(handle_submit)

In [ ]:
button = wid.Button(description="Calculate")
minutesText = wid.BoundedIntText(value=0,min=0,max=3060,step=1,description='Minutes:')
goalsText = wid.BoundedIntText(value=0,min=0,max=50,step=1,description='Goals:')
assistsText = wid.BoundedIntText(value=0,min=0,max=50,step=1,description='Assists:')
itemtext = [wid.HTML(value="Calculate expected goals + assists across an entire season of 3060 minutes, based on the amount of goals and assists achieved in the specified number of minutes.")]
items = [minutesText, goalsText, assistsText, button]
df = pd.DataFrame(np.array([[0, 0, 0]]), columns=['Min', 'Gls', 'Ast'])
answer = 0
answerText = wid.Text(value = str(answer), description='Goals + Assists')

In [ ]:
def on_button_clicked(b):
    print("Button clicked.")
    df = pd.DataFrame(np.array([[minutesText.value, goalsText.value, assistsText.value]]), columns=['Min', 'Gls', 'Ast'])
    answer = int(np.ceil(max(0, (model.predict(df) * 34))))
    if answer < (int(goalsText.value) + int(assistsText.value)):
        answer = (int(goalsText.value) + int(assistsText.value))
    answerText.value = str(answer)

button.on_click(on_button_clicked)

topbox = HBox(children=itemtext)
bottombox = HBox(children=items)
box = VBox(children=[topbox, bottombox, answerText])
box

In [ ]:
# df = pd.DataFrame(np.array([[minutesText.value, goalsText.value, assistsText.value]]), columns=['Min', 'Gls', 'Ast'])
# print(df)

In [ ]:
# # enter Mins, Gls, Ast, output estimated Gls+ast over 3060 mins

# df = pd.DataFrame(np.array([[2369, 7, 3]]), columns=['Min', 'Gls', 'Ast'])
# print(model.predict(df))
# answer = str(np.ceil(float((model.predict(df) * 34)))) 

# answerText = wid.Text(answer)
# display(answerText)
# # also add estimated point to a scatterplot?


In [ ]:
# model.get_params()
# #https://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/
# #tweak parameters
# from sklearn.model_selection import GridSearchCV
# alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
# grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas))
# grid.fit(X_train, y_train)
# print(grid)
# # summarize the results of the grid search
# print(grid.best_score_)
# print(grid.best_estimator_.alpha)

In [ ]:
predicted = model.predict(x_test)
predicted[predicted < 0] = 0
#predicted


In [ ]:
sns.scatterplot(x = dataset['Min'], y = dataset['GlsAst']);
sns.regplot(x = x_test['Min'], y = (predicted*34), color='g', scatter=False);


Here are the scores for the Bayesian Ridge model, starting with the r2, and then the mean squared error.

In [ ]:
predict_test = model.predict(x_test)
print(metrics.r2_score((y_test), predicted))
print(metrics.mean_squared_error((y_test), predicted))
# print(model.score(x_test, y_test))
